Trabalho feito por: Antonio Albuquerque, Davi César, João Ricardo.

# Imports


In [1]:
import pandas as pd
!pip -q install pyarrow
from google.colab import files
import numpy as np
import sys, os, json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report


# Importar CSV


In [2]:
uploaded = files.upload()
dataset = next(iter(uploaded))

Saving games.csv to games.csv


# Ler no pandas

In [27]:
df = pd.read_csv(dataset)
df.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [32]:
#mostrar campeoes mais pickados

df[[f"t1_champ{i}id" for i in range(1,6)] + [f"t2_champ{i}id" for i in range(1,6)]].melt(value_name="championId")["championId"].value_counts().head(10)


,count
championId,
412,13002
18,12983
67,10658
141,9853
64,9188
29,8838
40,8691
236,8300
202,8044


#Checar nulos e negativos


In [20]:
#negativos ou 0 de gameId
negativos = df[df["gameId"] <= 0]
len(negativos)


0

In [5]:
#Nulos de gameId
nulos = df[df["gameId"].isna()]
len(nulos)

0

In [6]:
#Nulos de gameId
nulos = df[df["creationTime"].isna()]
len(nulos)

0

In [7]:
#Nulos de gameId
nulos = df[df["gameDuration"].isna()]
len(nulos)

0

Agora, teremos uma validação de integridade para ver se está tudo realmente certo, como por exemplo : Cada partida deve ter 10 campeões válidos (5 de cada time), Cada time deve ter até 5 bans, Todos os feitiços usados devem existir no dicionário de spells.

#

In [8]:
# Conferir se cada partida tem exatamente 10 campeões
champ_cols = [c for c in df.columns if "champ" in c and "id" in c]
champ_count = df[champ_cols].notna().sum(axis=1)
print("Partidas com menos de 10 campeões:", (champ_count < 10).sum())

# Conferir se cada time tem no máximo 5 bans
ban_cols = [c for c in df.columns if "ban" in c]
ban_count = df[ban_cols].notna().sum(axis=1)
print("Partidas com mais de 10 bans:", (ban_count > 10).sum())



Partidas com menos de 10 campeões: 0
Partidas com mais de 10 bans: 0


Agora uma Validação de consistência, como por exemplo: gameDuration deve ser sempre positiva e > 300s (sem remakes). E se o time vencedor destruiu pelo menos 1 torre (obrigatório para vencer).



In [9]:
print("Partidas com duração < 300s:", (df["gameDuration"] < 300).sum())

# Coerência simples: time vencedor destruiu pelo menos 1 torre
df_winners = df[df["winner"] == 1]
print("Vitórias do time 1 sem torre destruída:", (df_winners["t1_towerKills"] == 0).sum())

df_winners2 = df[df["winner"] == 2]
print("Vitórias do time 2 sem torre destruída:", (df_winners2["t2_towerKills"] == 0).sum())

Partidas com duração < 300s: 1195
Vitórias do time 1 sem torre destruída: 631
Vitórias do time 2 sem torre destruída: 589


Primeiro, remover os Outliers.

In [10]:
df = df[df["gameDuration"] >= 300]


Agora, as partidas sem torres destruidas geram incosistência, em primeira instância, decidimos remover elas dos dados.

In [11]:
df = df[~((df["winner"] == 1) & (df["t1_towerKills"] == 0))]
df = df[~((df["winner"] == 2) & (df["t2_towerKills"] == 0))]

Agora uma Validação de unicidade, primeiro estamos vendo os gameIds duplicados e um exemplo de qual é e quantos são, depois mantemos apenas a primeira ocorrencia e apagamos as demais.

In [12]:
print("GameIDs duplicados:", df["gameId"].duplicated().sum())


GameIDs duplicados: 429


In [13]:
# Primeiro, pegar um dos gameId duplicados
game_id_duplicado = df["gameId"].value_counts()[df["gameId"].value_counts() > 1].index[0]

# Segundo, filtrar o DataFrame para mostrar todas as linhas com esse gameId
df[df["gameId"] == game_id_duplicado]


,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
1878,3331169499,1504634667164,1660,9,2,1,2,2,2,2,...,6,1,1,1,0,86,516,157,119,55
34092,3331169499,1504634667164,1660,9,2,1,2,2,2,2,...,6,1,1,1,0,86,516,157,119,55
34766,3331169499,1504634667164,1660,9,2,1,2,2,2,2,...,6,1,1,1,0,86,516,157,119,55


In [21]:
# Deixar apenas o primeiro gameID e apagar os iguais a ele, já que são 100% iguais.
df_sem_duplicatas = df.drop_duplicates(subset="gameId", keep="first")


In [30]:
# Dropar colunas indesejadas e deixar só os campeões pickados.
df = df[[f"t1_champ{i}id" for i in range(1, 6)] + [f"t2_champ{i}id" for i in range(1, 6)] + ["winner"]]


In [31]:
# Separar features e target
X = df.drop(columns=["winner"])
y = df["winner"]

# Split em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Treinar modelo Random Forest
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Predições
y_pred = model.predict(X_test)

# Avaliação
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Precisão:", precision_score(y_test, y_pred, average="weighted"))
print("F1-Score:", f1_score(y_test, y_pred, average="weighted"))

# Relatório completo (com precisão, recall e f1 por classe)
print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.5185472907360652
Precisão: 0.5181660300055552
F1-Score: 0.5175227508272467

Relatório de classificação:
              precision    recall  f1-score   support

           1       0.52      0.56      0.54      7823
           2       0.51      0.47      0.49      7624

    accuracy                           0.52     15447
   macro avg       0.52      0.52      0.52     15447
weighted avg       0.52      0.52      0.52     15447



PARTE EXTRA * av2 *


E por fim a Validação de enriquecimento (joins com dicionários), para verificar se os campeões e se as spells estão disponiveis corretamente no dicionário e existem.

In [17]:
from google.colab import files

uploaded = files.upload()

In [18]:
import json
import pandas as pd

# Abrir o JSON
with open("champion_info.json", "r", encoding="utf-8") as f:
    champ_info = json.load(f)

df_champions = pd.DataFrame.from_dict(champ_info["data"], orient="index")

# Conferir
print(df_champions.head())
print(df_champions.columns)


FileNotFoundError: [Errno 2] No such file or directory: 'champion_info.json'

In [ ]:
# Carregar campeões
with open("champion_info.json", "r", encoding="utf-8") as f:
    champ_info = json.load(f)
with open("champion_info_2.json", "r", encoding="utf-8") as f:
    champ_info2 = json.load(f)

# Transformar a chave "data" em DataFrame (cada campeão vira uma linha)
df_champions1 = pd.DataFrame.from_dict(champ_info["data"], orient="index")
df_champions2 = pd.DataFrame.from_dict(champ_info2["data"], orient="index")

# Concatenar os dois conjuntos de campeões
df_champions = pd.concat([df_champions1, df_champions2], ignore_index=True)

# Carregar spells
with open("summoner_spell_info.json", "r", encoding="utf-8") as f:
    spells_info = json.load(f)

df_spells = pd.DataFrame.from_dict(spells_info["data"], orient="index")

# Validar IDs de campeões
champ_cols = [c for c in df.columns if "champ" in c and "id" in c]
champ_ids = pd.unique(df[champ_cols].values.ravel())

champ_dict_ids = pd.to_numeric(df_champions["key"], errors="coerce").dropna().astype(int).unique()

invalid_champs = [c for c in champ_ids if c not in champ_dict_ids]
print("IDs de campeões sem correspondência:", invalid_champs)

# Validar IDs de feitiços
spell_cols = [c for c in df.columns if "sum" in c]
spell_ids = pd.unique(df[spell_cols].values.ravel())

# Converter somente os numéricos
spell_dict_ids = pd.to_numeric(df_spells["key"], errors="coerce").dropna().astype(int).unique()

invalid_spells = [s for s in spell_ids if s not in spell_dict_ids]
print("IDs de feitiços sem correspondência:", invalid_spells)

